In [3]:
pip install matplotlib==3.10.7 seaborn==0.13.2

  Obtaining dependency information for matplotlib==3.10.7 from https://files.pythonhosted.org/packages/31/3c/80816f027b3a4a28cd2a0a6ef7f89a2db22310e945cd886ec25bfb399221/matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata
  Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for seaborn==0.13.2 from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/19/e8/6026ed58a64563186a9ee3f29f41261fd1828f527dd93d33b60feca63352/contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install scikit-learn==1.7.2

  Obtaining dependency information for scikit-learn==1.7.2 from https://files.pythonhosted.org/packages/c6/99/ed35197a158f1fdc2fe7c3680e9c70d0128f662e1fee4ed495f4b5e13db0/scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata
  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for scipy>=1.8.0 from https://files.pythonhosted.org/packages/c2/7f/acbd28c97e990b421af7d6d6cd416358c9c293fc958b8529e0bd5d2a2a19/scipy-1.16.3-cp312-cp312-win_amd64.whl.metadata
  Using cached scipy-1.16.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/1e/e8/685f47e0d754320684db4425a0967f7d3fa70126bffd76110b7009a0090f/joblib-1.5.2-py3-none-any.whl.metadata
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b3


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns 

In [2]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [3]:
df = pd.read_csv(r"../stackoverflow_trends_data.csv")
df.head(5)

,surrogate_key,creation_full_date,year,month,quarter,day_of_week,month_name,day_name,date_key,tag,score,view_count,favorite_count,comment_count,answer_count,answer_status
0,-2937206958259993047,2020-05-01 00:00:00.000,2020,5,2,6,May,Friday,20200501,python,0,47,NaN,0,1,answered
1,2708228319434441337,2020-05-03 00:00:00.000,2020,5,2,1,May,Sunday,20200503,python,0,34,NaN,0,1,answered
2,2708228319434441337,2020-05-03 00:00:00.000,2020,5,2,1,May,Sunday,20200503,python,0,73,NaN,0,1,answered
3,-7128293146829867177,2020-05-04 00:00:00.000,2020,5,2,2,May,Monday,20200504,python,0,405,NaN,0,1,answered
4,-7128293146829867177,2020-05-04 00:00:00.000,2020,5,2,2,May,Monday,20200504,python,0,58,NaN,0,1,answered


In [4]:
df.shape

(4718839, 16)

In [5]:
df.isnull().sum()

surrogate_key               0
creation_full_date          0
year                        0
month                       0
quarter                     0
day_of_week                 0
month_name                  0
day_name                    0
date_key                    0
tag                      1182
score                       0
view_count                  0
favorite_count        4056076
comment_count               0
answer_count                0
answer_status               0
dtype: int64

In [6]:
# 1. Create the LabelEncoder
le = LabelEncoder()

# 2. Fit and Transform
# We cast to str to ensure LabelEncoder handles the NaNs as a category 'nan' temporarily
df["tag_enc"] = le.fit_transform(df["tag"].astype(str))

In [7]:
# 3. THE CRITICAL FIX: Mask the missing values back to np.nan
# Find where the original data was null
missing_mask = df["tag"].isnull() 

# Force those indices to be np.nan in the encoded column
# This tells KNNImputer: "Hey, this is actually missing data!"
df.loc[missing_mask, "tag_enc"] = np.nan

In [8]:
# 4. Scaling (KNN relies on distance, so scaling is good)
scaler = MinMaxScaler()
# Reshape is needed for single feature scaling
df["tag_scaled"] = scaler.fit_transform(df[["tag_enc"]])

In [13]:
# 5. Impute
imputer = KNNImputer(n_neighbors=5) # n_neighbors=5 is usually safer than 10 for categorical voting
imputed_matrix = imputer.fit_transform(
    df[["tag_scaled", "score", "view_count", "answer_count"]]
)

# Update the scaled column with imputed values
df["tag_scaled"] = imputed_matrix[:, 0]

KeyboardInterrupt: 

In [ ]:
# 6. Inverse Scaling
# Reshape back for inverse transform
df["tag_enc_imputed"] = scaler.inverse_transform(df[["tag_scaled"]])

In [ ]:
# 7. Rounding and Inverse Label Encoding
# We must round to the nearest integer because LabelEncoder expects integers
df["tag_enc_imputed"] = df["tag_enc_imputed"].round().astype(int)

# Clip values to ensure they don't exceed the LabelEncoder's range (safety check)
df["tag_enc_imputed"] = df["tag_enc_imputed"].clip(0, len(le.classes_) - 1)

# Get final strings
df["tag_final"] = le.inverse_transform(df["tag_enc_imputed"])

3295       NaN
3498       NaN
10015      NaN
16285      NaN
24811      NaN
          ... 
4692439    NaN
4701943    NaN
4705857    NaN
4711042    NaN
4713193    NaN
Name: tag_final, Length: 1182, dtype: object

In [ ]:
# 8. Verify
print(f"Remaining NaNs: {df['tag_final'].isna().sum()}")

tag_final
<class 'str'>      4717657
<class 'float'>       1182
Name: count, dtype: int64